In [64]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import numpy as np
import glob
import import_ipynb as ipynb
import n_most_similar_games as matchup
from bs4 import BeautifulSoup
from collections import Counter

CAL_OFF = 100
OPP_DEF = 40
#NOTE: EPV CAPS ROUGHLY AT 5 BC EXTRA POINT IS NOT ACCOUNTED FOR?? BRING UP AT MEETING


In [41]:
temp = []
data = []
subpxp = []
for year in range(2008,2018):    
    path ='NCAA-Football-Data/' + str(year) + 'PXP' # use your path
    allFiles = glob.glob(path + "/*.csv")
    frame = pd.DataFrame()
    list_ = []
    cols = ['gameId', 'driveIndex', 'year', 'week', 'clock', 'quarter', 'down', 'yardLine', 'distance', 'yardsGained',
            'offenseTeam', 'defenseTeam', 'description', 'type',
            'homeScore', 'awayScore', 'homeTeam', 'awayTeam']
    for file_ in allFiles:
        df = pd.read_csv(file_,index_col=None, header=0, usecols=cols)
        list_.append(df)
        subpxp = pd.concat(list_, ignore_index=True)
    temp.append(subpxp)
    data= pd.concat(temp, ignore_index=True)

In [42]:
import cal_football as cf


In [43]:
pxp = cf.extract_data(data)
badyard = ~((pxp['yrdline100']>=100)|(pxp['yrdline100']<=0))
# possession_values = cf.get_possession_value(ko_and_pos)

In [ ]:
game_data, rating_data, pxp_data = matchup.get_ratings()
games_compared, matchup_pxp = matchup.most_similar(off_rating = CAL_OFF, def_rating = OPP_DEF, n = 100, game_data = game_data, rating_data = rating_data, pxp_data=pxp_data)

#matchup_pxp = cf.extract_data(matchup_pxp)
#matchup_pxp

/Users/jeremylevitt/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/jeremylevitt/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [ ]:
matchup_pxp = cf.m_extract_data(matchup_pxp)

In [ ]:
def modified_extract_k_and_p(pxp, game_data):
    
    
    ko_mask = pxp['type'].isin(['Kickoff', 'Kickoff Return (Defense)','Kickoff Return Touchdown','Kickoff Return (Offense)'])  
    kickoffs = pxp.loc[ko_mask]
    # Exclude kickoffs and XPs and groupby GameID and Drive
    xp_mask = pxp['type'].isin(['Extra Point Good','Blocked PAT','Extra Point Missed','No Play'])
    game_drives = pxp.loc[~ko_mask & ~xp_mask].groupby(['gameId', 'driveIndex'])

    poss_starts = game_drives.head(1)
    
    

    # Concatenate kickoffs and first plays, sort, and reindex
    ko_and_poss = pd.concat([kickoffs, poss_starts])\
            .sort_values(['gameId', 'driveIndex', 'index'], na_position='first')
    ko_and_poss.reset_index(drop=True, inplace=True)
    # Extract game halves
    game_halves = ko_and_poss.groupby(['gameId', 'half'])
    
    
    #Original Score_change explanaition:
    # Compute changes in the scores.  + for Home and - for Away. Aka if the away team score then -7
    
    #Modified: Negative score change if team analogous to Cal's opponent scores. Postitive if team analagous to Cal scores
    #See n_most_similar most_similar() function for explaination as to what matchup_list[3] is.
    print(str(len(game_halves)))
    # Backfill the score change so that each 
    score_change_arr = []
    print("length of pxp: " + str(len(pxp)))
    score_change = game_halves['homeScore'].diff() - game_halves['awayScore'].diff()
    ko_and_poss['unfinalized_sc'] = score_change
    for index, row in ko_and_poss.iterrows():
        if row['home_off_away_def']:
            score_change_arr.append(row['unfinalized_sc'])
        else:
            score_change_arr.append(-row['unfinalized_sc'])
    ko_and_poss['score_change'] = score_change_arr
    print("Length of KO: " + str(len(ko_and_poss)))
    print("\nLength of Score_Change_arr: " + str(len(score_change_arr)))
    ko_and_poss['score_change'] = score_change_arr
    ko_and_poss['NextScore'] = score_change.fillna(0).replace(to_replace=0., method='bfill')
#     ko_and_poss.loc[ko_and_poss['Touchdown']==1, 'NextScore'] = ko_and_poss['NextScore']/ko_and_poss['NextScore']*7
    
    #originial algorithm:
    # Determine if the possessing team is home or away
    # NextScore is unchanged if posteam == hometeam and negated if posteam == awayteam
    
    #Matchup Modified Algorithm:
    #Determine if the posessing team is analagous to Cal or analgous to opponent team
    #NextScore is unchanged if posteam == ANALAGOUSCAL and negated if posteam == ANALGOUSOPP
    
    posteam_is_CAL_arr = []
    posteam_is_OPP_arr = []
    for index, row in ko_and_poss.iterrows():
        if row["home_off_away_def"]:
            posteam_is_CAL_arr.append(int(row['offenseTeam'] == row['homeTeam']))
            posteam_is_OPP_arr.append(int(row['offenseTeam'] == row['awayTeam']))
        else:
            posteam_is_CAL_arr.append(int(row['offenseTeam'] == row['awayTeam']))
            posteam_is_OPP_arr.append(int(row['offenseTeam'] == row['homeTeam']))
     
    ko_and_poss['posteam_is_CAL'] = posteam_is_CAL_arr
    ko_and_poss['posteam_is_OPP'] = posteam_is_OPP_arr
    ko_and_poss['PossessionValue'] = ko_and_poss['NextScore'] * \
        (ko_and_poss['posteam_is_CAL'] - ko_and_poss['posteam_is_OPP'])

    return ko_and_poss

In [47]:
ko_and_poss = cf.extract_kickoffs_and_possessions(pxp)

In [48]:
fgs = cf.extract_field_goals(pxp)
third_downs = cf.extract_third_downs(pxp)
punts = cf.extract_punts(pxp)

ekv,epv_model = cf.epv_model(ko_and_poss)
exp_conv_pct_model = cf.exp_conv_pct_model(third_downs)
exp_net_punt_dist_model = cf.exp_net_punt_dist_model(punts)
exp_fg_prob_model = cf.exp_fg_prob_model(fgs)

In [ ]:
m_ko_and_poss = modified_extract_k_and_p(matchup_pxp, game_data)
m_third_downs = cf.extract_third_downs(matchup_pxp)
m_ko_and_poss

In [62]:
m_ekv, m_epv_model = cf.epv_model(m_ko_and_poss)
m_exp_conv_pct_model = cf.exp_conv_pct_model(m_third_downs)


In [63]:
m_epv_model


yrdline100
1     1.043728
2     1.028766
3     1.013805
6     0.968920
7     0.953958
8     0.938996
9     0.924035
11    0.894111
12    0.879150
14    0.849226
15    0.834265
16    0.819303
17    0.804341
19    0.774418
20    0.759456
22    0.729533
23    0.714571
24    0.699609
25    0.684648
26    0.669686
27    0.654724
28    0.639763
29    0.624801
30    0.609839
31    0.594878
32    0.579916
33    0.564954
34    0.549993
35    0.535031
36    0.520069
        ...   
70    0.011372
71   -0.003589
72   -0.018551
73   -0.033513
74   -0.048474
75   -0.063436
76   -0.078398
77   -0.093359
78   -0.108321
79   -0.123283
80   -0.138244
81   -0.153206
82   -0.168168
83   -0.183129
84   -0.198091
85   -0.213053
86   -0.228014
87   -0.242976
88   -0.257938
89   -0.272899
90   -0.287861
91   -0.302823
92   -0.317785
93   -0.332746
94   -0.347708
95   -0.362670
96   -0.377631
97   -0.392593
98   -0.407555
99   -0.422516
Name: EPV, Length: 93, dtype: float64

In [55]:
epv_model

yrdline100
0      5.054143
1      4.999428
2      4.944714
3      4.889999
4      4.835284
5      4.780569
6      4.725854
7      4.671140
8      4.616425
9      4.561710
10     4.506995
11     4.452280
12     4.397566
13     4.342851
14     4.288136
15     4.233421
16     4.178706
17     4.123992
18     4.069277
19     4.014562
20     3.959847
21     3.905132
22     3.850418
23     3.795703
24     3.740988
25     3.686273
26     3.631558
27     3.576844
28     3.522129
29     3.467414
         ...   
73     1.059963
74     1.005248
75     0.950533
76     0.895818
77     0.841103
78     0.786389
79     0.731674
80     0.676959
81     0.622244
82     0.567529
83     0.512815
84     0.458100
85     0.403385
86     0.348670
87     0.293955
88     0.239241
89     0.184526
90     0.129811
91     0.075096
92     0.020381
93    -0.034333
94    -0.089048
95    -0.143763
96    -0.198478
97    -0.253193
98    -0.307907
99    -0.362622
100   -0.417337
159   -3.645510
162   -3.809655
Name: EPV, Le

In [52]:
epv_model.to_pickle("./epv_model.pkl")
exp_conv_pct_model.to_pickle("./exp_conv_pct_model.pkl")
exp_net_punt_dist_model.to_pickle("./exp_net_punt_dist_model.pkl")
exp_fg_prob_model.to_pickle("./exp_fg_prob_model.pkl")


In [53]:
m_epv_model.to_pickle("./m_epv_model.pkl")
m_exp_conv_pct_model.to_pickle("./m_exp_conv_pct_model.pkl")

In [54]:
def build_decision_maker(yrdline100, ydstogo, ekv, epv_model, exp_conv_pct_model, 
                         exp_net_punt_dist_model, exp_fg_prob_model,
                        m_ekv, m_epv_model_use, m_exp_conv_pct_model, w):
    
    YRDSTOGO_CAP = 10  # Model restriction
    PUNT_LIM = 29      # Model restriction
    FG_LIM = 63        # Model restriction
    FG_OFFSET = 8      # Field goal placement distance (not including the 10 for the endzone)
    FG_FAIL_RESET = 0  # Where the ball is placed after a failed field goal.  If 0, at the spot.  
                       # Otherwise, at the spot or at the reset spot, whichever is better.
    if yrdline100 < 10:
        region = 'Inside10'
    elif yrdline100 < 20:
        region = '10to20'
    else:
        region = 'Beyond20'
            
        # 1. Expected value of going for it
        
        # Value of failing (approximately turning over at same spot)
    conv_fail_yrdline100 = 100 - yrdline100
    conv_fail_epv = -epv_model[conv_fail_yrdline100] 
        
    m_conv_fail_epv = -m_epv_model[conv_fail_yrdline100]
        
        # Value of converting (approximately at the first down marker)
    if yrdline100 == ydstogo:
        conv_succ_epv = 6 - ekv
        m_conv_succ_epv = 6 - m_ekv
    else:
        first_down_yrdline100 = yrdline100 - ydstogo
        conv_succ_epv = epv_model[first_down_yrdline100] -1
        m_conv_succ_epv = m_epv_model[first_down_yrdline100] - 1

        # Conversion Pct
    exp_conv_pct = exp_conv_pct_model[(region, ydstogo)] if ydstogo < YRDSTOGO_CAP else None
    m_exp_conv_pct = m_exp_conv_pct_model[(region, ydstogo)] if ydstogo < YRDSTOGO_CAP else None
        
        # Overall expected value of going for it
    unmod_go_ev = exp_conv_pct * conv_succ_epv + (1 - exp_conv_pct) * conv_fail_epv
    m_go_ev = m_exp_conv_pct * m_conv_succ_epv + (1 - m_exp_conv_pct) * m_conv_fail_epv
    go_ev = w * m_go_ev + (1 - w) * unmod_go_ev
        ####THIS IS WHERE YOU ADD THE MATCHUP EPV WEIGHT AND STUFF (ABOVE)
        
        
        
        
        # 2. Expected value of punting
        
        # Expected next yardline
    if yrdline100 >= PUNT_LIM:
        exp_net_punt_dist = exp_net_punt_dist_model[yrdline100]
        exp_yrdline100 = 100 - yrdline100 + int(exp_net_punt_dist)
            
            # Overall expected value of punting
        punt_ev = -epv_model[exp_yrdline100]
    else:
        punt_ev = None
            
        # 3. Expected value of kicking a field goal
    fg_dist =  yrdline100 + 10 + FG_OFFSET
    if fg_dist <= FG_LIM:
        # Probability of success
        exp_fg_prob = exp_fg_prob_model[fg_dist ]

        # Expected value of field success
        fg_succ_epv = 3 - ekv

        # EPV of field goal fail
        fg_fail_yrdline100 = 100 - yrdline100 - FG_OFFSET
        fg_fail_epv = -epv_model[fg_fail_yrdline100]

        # Overall expected value kicking
        fg_ev = fg_succ_epv * exp_fg_prob + fg_fail_epv * (1 - exp_fg_prob)
    else:
        fg_ev = None
    
    choices = [('go for it', go_ev), ('punt', punt_ev), ('kick', fg_ev)]
    max_val = -100
    for choice, ev in choices:
        if ev is None:
            continue
        if ev > max_val:
            max_val = ev
            decision = choice
    
    print_message=False
    if print_message:
        print("Expected Values")
        print("Go for it: {:.2f}".format(go_ev))
        if punt_ev is not None:
            print("Punt: {:.2f}".format(punt_ev))
        else:
            print("Punt: TOO CLOSE TO PUNT")
        if fg_ev is not None:
            print("FG: {:.2f}".format(fg_ev))
        else:
            print("FG: TOO FAR TO KICK")        

        print()
        print("Coach, you should:")
        print(decision.upper())
    
    return decision

# Build the decision maker

yrdlines = list(range(1, 100))
down_dist = list(range(1, 10))

decisions = []
for yrdstogo in down_dist:
    tmp = []
    for yrdline in yrdlines:
        if (yrdline >= yrdstogo) and (100 - yrdline + yrdstogo >= 10): # Exclude impossible scenarios
            decision = build_decision_maker(yrdline, yrdstogo, 
                                            ekv, epv_model, exp_conv_pct_model, exp_net_punt_dist_model, 
                                            exp_fg_prob_model, m_ekv, m_epv_model, m_exp_conv_pct_model, 0.05)
            tmp.append(decision)
        else:
            tmp.append(None)
    decisions.append(tmp)
# Convert to a table
pd.DataFrame(decisions, columns=yrdlines, index=down_dist)

import numpy as np
data = np.zeros((9, 99))
for i in range(9):
    for j in range(99):
        decision = decisions[i][j]
        if decision == 'go for it':
            val = 1
        elif decision == 'punt':
            val = 2
        elif decision == 'kick':
            val = 3
        elif decision is None:
            val = np.nan
        data[i, j] = val
data = data[::-1,::-1]

fig, ax = plt.subplots(figsize=(12, 9))

ax.pcolor(
    data,
    vmin=1,
    vmax=8,
    edgecolors='k', 
    linewidths=.2,
    cmap=plt.get_cmap('Dark2'),
    alpha=.6
);
ax.set_xticks([0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
ax.set_xticklabels(['Own Goal', '90', '80', '70', '60', '50', '40', '30', '20', '10', 'Goal'])
ax.set_yticks([
    yd - .5
    for yd in range(1, 10)
])
ax.set_yticklabels([
    '4th and {}'.format(yd)
    for yd in range(9, 0, -1)
])
ax.text(50, 6, 'Go for it', size=25)
ax.text(20, 1.5, 'Punt', size=25)
ax.text(75, 1.5, 'Kick', size=25)
ax.tick_params(axis='both', which='major', labelsize=15);

KeyError: 4